In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
%load_ext autoreload
%autoreload 2
plt.style.use('ggplot')

2021-12-14 16:50:33.606981: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
W1214 16:50:45.215559 47430702768960 __init__.py:333] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1214 16:50:45.272868 47430702768960 __init__.py:333] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1214 16:50:45.274479 47430702768960 __init__.py:333] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1214 16:50:45.429379 47430702768960 __init__.py:366] Limited tf.summary API due to missing TensorBoard installation.


In [2]:
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
import sys 
sys.path.append("/scratch/izar/kapps/DEX-Cyclic-Arbitrage/")
from config.get import cfg

# Data loading & preprocessing

In [4]:
data = pd.read_csv(cfg['directories']['preprocessed_data'])
data.head()

,cycle_id,token1,token2,baseAmount,quoteAmount,quotePrice,gasPrice,gasValue,time
0,0,WETH,SHUF,0.010000,10.620910,1062.091006,34.100011,0.009516,2020-05-18 17:48:23
1,0,WETH,SHUF,0.009372,10.000000,1067.003380,33.210001,0.006049,2020-05-18 17:59:48
2,0,WETH,SHUF,0.010000,10.592990,1059.298991,33.100009,0.005471,2020-05-18 18:01:30
3,0,WETH,SHUF,0.010000,10.143472,1014.347243,33.100009,0.007748,2020-05-18 18:04:30
4,0,WETH,SHUF,0.009804,10.000000,1020.026068,33.100009,0.006029,2020-05-18 18:04:54


In [5]:
#!pip install sklearn

## Log scaling

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from data exploration log was found to be a good normaliser
def log_scaling(data):
    data['baseAmount']  = np.log(data.baseAmount)
    data['quoteAmount'] = np.log(data.quoteAmount)
    data['quotePrice']  = np.log(data.quotePrice)
    return data;

log_data = log_scaling(data)

## Transform `tokens` into dummy variables (one hot encoding)

In [7]:
# transform tokens into dummies
from sklearn.preprocessing import OneHotEncoder
token_columns = list(filter(lambda c: c.startswith('token'),log_data.columns))
def one_hot_tokens():
    one_enc = OneHotEncoder(sparse=False) 
    unique_tokens = np.unique(pd.concat([log_data[token] for token in token_columns],axis=0))
    one_enc.fit(unique_tokens.reshape(-1, 1))
    # transform
    encode = lambda col: one_enc.transform(log_data[col].to_numpy().reshape(-1, 1))
    # encode and convert as dataframes
    
    encoded = [pd.DataFrame(encode(token)).add_prefix(f"token_{key}_") for key,token in enumerate(token_columns)]
    return pd.concat(encoded, axis='columns')

In [8]:
X = pd.concat([log_data.drop(columns=token_columns),one_hot_tokens()], axis='columns')

In [12]:
X.head()

,cycle_id,baseAmount,quoteAmount,quotePrice,gasPrice,gasValue,time,token_0_0,token_0_1,token_0_2,...,token_1_84,token_1_85,token_1_86,token_1_87,token_1_88,token_1_89,token_1_90,token_1_91,token_1_92,token_1_93
0,0,-4.605170,2.362825,6.967995,34.100011,0.009516,2020-05-18 17:48:23,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,-4.670024,2.302585,6.972609,33.210001,0.006049,2020-05-18 17:59:48,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,-4.605170,2.360192,6.965363,33.100009,0.005471,2020-05-18 18:01:30,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,-4.605170,2.316830,6.922001,33.100009,0.007748,2020-05-18 18:04:30,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,-4.624998,2.302585,6.927583,33.100009,0.006029,2020-05-18 18:04:54,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#  Transform each cycle into a feature matrix  & Zero-padding

In [79]:
N_TOKEN = 3 # cycle length
K = 2       # quote price & gasPrice
N = 1000    # number of cycles
P = 600     # max time series length per cycle

def build_tensor(data):
    tensor = np.zeros((N, N_TOKEN,P, K))
    i = 0
    def get_sorted_token_map(g):
        t = g[['token1','token2']].values
        u, ind = np.unique(t, return_index=True)
        u_sorted =  u[np.argsort(ind)]
        return dict(zip(u_sorted, range(len(u_sorted))))

    for _, group in iter(data.groupby('cycle_id')):
        token_map = get_sorted_token_map(group)
        for _, g in iter(group.groupby(['token1','token2'])):
            a = g[['quotePrice','gasPrice']].values
            # zero padding
            padded = np.pad(a, [(0, P - len(a)),(0,0)])
            # assign and reshape into a matrix
            first_token = g.token1.iloc[0]
            token_ind = token_map[first_token]
            tensor[i,token_ind,:,:] = padded.reshape(1,P,K)
        i = i+1
    return tensor
    
X = build_tensor(log_data)

In [91]:
a = np.array([[[0,1],[2,3]],[[4,5],[6,7]],[[8,9],[10,11]],[[12,13],[14,15]]])
a

array([[[ 0,  1],
        [ 2,  3]],

       [[ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11]],

       [[12, 13],
        [14, 15]]])

In [114]:
# train/test split and standard scaling 
X_train, X_test = train_test_split(X, test_size=0.3, random_state=123)


#scaler = StandardScaler()
#scaler.fit(X_train)

#X_train_scaled = scaler.transform(X_train)
#X_test_scaled  = scaler.transform(X_test)

X_train_scaled = X_train
X_test_scaled = X_test

# Model definition

In [115]:
def build_model():
    input_img = keras.Input(shape=(3,600, 2))
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((3, 3), padding='same')(x)
    x = layers.Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    encoded = layers.Conv2D(1, (2, 2), activation='relu', padding='same')(x)
    # at this point the representation is (4, 4, 8) i.e. 128-dimensional
    x = layers.Conv2D(4, (2, 2), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((1, 2))(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((3, 3))(x)
    decoded = layers.Conv2D(2, (3, 3), activation='relu', padding='same')(x)

    autoencoder = keras.Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    
    return autoencoder

In [116]:
autoencoder = build_model()
autoencoder.summary()

Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 3, 600, 2)]       0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 3, 600, 8)         152       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 1, 200, 8)         0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 1, 200, 4)         292       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 1, 100, 4)         0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 1, 100, 1)         17        
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 1, 100, 4)       

In [119]:
autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=10,
                batch_size=32,
                shuffle=True,
                verbose=0)